In [1]:
from EmbeddingFunctions import *
TOKENIZERS_PARALLELISM=False

In [2]:
rag, metadata = initialize_multimodal_vector_database()
#metadata = add_NLMCXR_to_vectorstore(rag)
llm, processor  = create_generator_model()

{'model_name': 'vidore/colpali', 'full_document_collection': True, 'highest_doc_id': 56, 'library_version': '0.0.2.post2'}


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ccig/miniconda3/envs/rag/lib/python3.9/site-packages/byaldi/colpali.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.extend(torch.load(fil

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
message= 'If the cardiac silhouette and mediastinum size are within normal limits, is there evidence of pneumothorax?'
retrieved = retrieve_similar_report(rag, message, 1)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [4]:
retrieved

[{'doc_id': 51, 'page_num': 1, 'score': 29.875, 'metadata': {}, 'base64': '...'}]

In [5]:
metadata[51]

{'file_name': 'CXR1'}

In [6]:
image = convert_from_path(os.path.join('/home/ccig/Desktop/Nuno/rag_data/NLMCXR_pdf_micro', f'{metadata[retrieved[0]["doc_id"]]["file_name"]}.pdf'))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
image = convert_from_path(os.path.join('/home/ccig/Desktop/Nuno/rag_data/NLMCXR_pdf_micro', f'{metadata[retrieved[0]["doc_id"]]["file_name"]}.pdf'))

image_index = retrieved[0]["page_num"] - 1

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image[image_index],
            },
            {"type": "text", "text": message},
        ],
    }
]
    

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda:3")
    

generated_ids = llm.generate(**inputs, max_new_tokens=50)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
output_text

['No, there is no evidence of pneumothorax.']